In [ ]:
import tensorflow as tf
import os
try:
  tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print(f'Running on a TPU w/{tpu_cluster_resolver.num_accelerators()["TPU"]} cores')
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please make sure you have successfully chosen TPU runtime from the Edit/Notebook settings menu')


Running on a TPU w/8 cores


In [ ]:
strategy = tf.distribute.TPUStrategy(tpu_cluster_resolver) #분산처리 시 이 부분 코드가 바뀜 처리 전략을 설정

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import numpy as np

# 데이터셋 클래스 정의
class JointAngleDataset(Dataset):
    def __init__(self, angle_differences, feedbacks, tokenizer, max_length):
        self.angle_differences = angle_differences
        self.feedbacks = feedbacks
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.angle_differences)

    def __getitem__(self, idx):
        angle_str = ' '.join([f"{angle:.2f}" for angle in self.angle_differences[idx]])
        text = f"Angle differences: {angle_str} Feedback: {self.feedbacks[idx]}"
        encoding = self.tokenizer(text,
                                  truncation=True,
                                  max_length=self.max_length,
                                  padding='max_length',
                                  return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

# 가상의 데이터 생성 (더 다양한 피드백 추가)
x = [45.71, 32.08, 23.86, 16.81, 14.44, -21.22, 20.13, 10.23]
array_x = np.array(x)
angle_differences = np.tile(array_x, (1000, 1))  # 데이터 수를 늘림

feedbacks = [
    "왼쪽 팔꿈치와 오른쪽 무릎, 왼쪽 무릎을 더 구부리세요.",
    "왼쪽 팔꿈치를 더 펴시고, 오른쪽 발목을 살짝 구부려주세요.",
    "오른쪽 무릎을 더 구부리시고, 왼쪽 발목을 펴세요.",
    "오른쪽 팔꿈치를 더 구부리세요.",
    "왼쪽 발목과 오른쪽 무릎을 살짝 더 구부려주세요.",
    # ... 더 많은 피드백 (다양하게 추가)
] * 200  # 데이터를 충분히 다양하게

# GPT-2 토크나이저 및 모델 로드
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')

# 데이터셋 및 데이터로더 생성
max_length = 128
dataset = JointAngleDataset(angle_differences, feedbacks, tokenizer, max_length)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 에폭 및 스케줄러 설정
num_epochs = 10  # 학습 에폭 수를 늘림
num_training_steps = num_epochs * len(train_dataloader)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# 학습 루프
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average training loss: {avg_train_loss:.4f}")

    # 검증
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Validation loss: {avg_val_loss:.4f}")

# 모델 저장
model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")

# 새로운 각도 차이에 대한 피드백 생성 함수
def generate_feedback(model, tokenizer, angle_differences, max_length=50):
    model.eval()
    angle_str = ' '.join([f"{angle:.2f}" for angle in angle_differences])
    input_text = f"Angle differences: {angle_str} Feedback:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, num_return_sequences=1,
                                do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text.split("Feedback:")[1].strip()

# 새로운 각도 차이에 대한 피드백 생성 예시
new_angle_differences = np.random.randn(8)
feedback = generate_feedback(model, tokenizer, new_angle_differences)
print(f"New angle differences: {new_angle_differences}")
print(f"Generated feedback: {feedback}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Average training loss: 0.6207
Validation loss: 0.0281
Epoch 2/10, Average training loss: 0.0494
Validation loss: 0.0156
Epoch 3/10, Average training loss: 0.0350
Validation loss: 0.0130
Epoch 4/10, Average training loss: 0.0285
Validation loss: 0.0129
Epoch 5/10, Average training loss: 0.0265
Validation loss: 0.0129
Epoch 6/10, Average training loss: 0.0235
Validation loss: 0.0130
Epoch 7/10, Average training loss: 0.0230
Validation loss: 0.0133
Epoch 8/10, Average training loss: 0.0220
Validation loss: 0.0127
Epoch 9/10, Average training loss: 0.0220
Validation loss: 0.0128
Epoch 10/10, Average training loss: 0.0218
Validation loss: 0.0127


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


New angle differences: [ 0.47284168  1.46294986 -0.18795335 -0.67921202 -1.60387553  0.370953
 -0.07279078 -0.41565269]
Generated feedback: 오른쪽 무릎


In [ ]:
np.random.randn(1000, 8)

array([[ 1.34736637, -1.8143916 ,  0.04704188, ...,  0.35583652,
        -0.21740608, -0.46065274],
       [-0.10770699, -0.98235335, -1.46805236, ..., -0.92416535,
        -1.87038028, -0.5954919 ],
       [ 1.63013404, -0.88823028,  0.50752091, ..., -1.90075033,
        -1.49807952,  0.13376651],
       ...,
       [ 1.57986496, -1.11977729, -0.15888714, ..., -1.13550211,
        -1.34414046,  0.85094211],
       [-0.8520548 , -1.85883056,  0.09253962, ...,  1.23424007,
        -1.49521018,  1.45920281],
       [ 0.86597681,  2.00353568, -0.60138822, ..., -1.95104412,
         0.71029575,  2.00843286]])

In [ ]:
array_x=np.array(x)

In [ ]:
np.tile(array_x, (500, 1))

array([[ 43.97738345,  32.63481825,  22.8612839 , ..., -20.8108553 ,
         18.29103916,   6.93029839],
       [ 43.97738345,  32.63481825,  22.8612839 , ..., -20.8108553 ,
         18.29103916,   6.93029839],
       [ 43.97738345,  32.63481825,  22.8612839 , ..., -20.8108553 ,
         18.29103916,   6.93029839],
       ...,
       [ 43.97738345,  32.63481825,  22.8612839 , ..., -20.8108553 ,
         18.29103916,   6.93029839],
       [ 43.97738345,  32.63481825,  22.8612839 , ..., -20.8108553 ,
         18.29103916,   6.93029839],
       [ 43.97738345,  32.63481825,  22.8612839 , ..., -20.8108553 ,
         18.29103916,   6.93029839]])

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import numpy as np

# 데이터셋 클래스 정의
class JointAngleDataset(Dataset):
    def __init__(self, angle_differences, feedbacks, tokenizer, max_length):
        self.angle_differences = angle_differences
        self.feedbacks = feedbacks
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.angle_differences)

    def __getitem__(self, idx):
        angle_str = ' '.join([f"{angle:.2f}" for angle in self.angle_differences[idx]])
        text = f"Angle differences: {angle_str} Feedback: {self.feedbacks[idx]}"
        encoding = self.tokenizer(text,
                                  truncation=True,
                                  max_length=self.max_length,
                                  padding='max_length',
                                  return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

# 가상의 데이터 생성 (더 다양한 피드백 추가)
x = [45.71, 32.08, 23.86, 16.81, 14.44, -21.22, 20.13, 10.23]
array_x = np.array(x)
angle_differences = np.tile(array_x, (1000, 1))  # 데이터 수를 늘림

feedbacks = [
    "왼쪽 팔꿈치와 오른쪽 무릎, 왼쪽 무릎을 더 구부리세요.",
    "왼쪽 팔꿈치를 더 펴시고, 오른쪽 발목을 살짝 구부려주세요.",
    "오른쪽 무릎을 더 구부리시고, 왼쪽 발목을 펴세요.",
    "오른쪽 팔꿈치를 더 구부리세요.",
    "왼쪽 발목과 오른쪽 무릎을 살짝 더 구부려주세요.",
    # ... 더 많은 피드백 (다양하게 추가)
] * 200  # 데이터를 충분히 다양하게

# GPT-2 토크나이저 및 모델 로드
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')

# 데이터셋 및 데이터로더 생성
max_length = 128
dataset = JointAngleDataset(angle_differences, feedbacks, tokenizer, max_length)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 에폭 및 스케줄러 설정
num_epochs = 10  # 학습 에폭 수를 늘림
num_training_steps = num_epochs * len(train_dataloader)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# 학습 루프
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average training loss: {avg_train_loss:.4f}")

    # 검증
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Validation loss: {avg_val_loss:.4f}")

# 모델 저장
model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")

# 새로운 각도 차이에 대한 피드백 생성 함수
def generate_feedback(model, tokenizer, angle_differences, max_length=50):
    model.eval()
    angle_str = ' '.join([f"{angle:.2f}" for angle in angle_differences])
    input_text = f"Angle differences: {angle_str} Feedback:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, num_return_sequences=1,
                                do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text.split("Feedback:")[1].strip()

# 새로운 각도 차이에 대한 피드백 생성 예시
new_angle_differences = np.random.randn(8)
feedback = generate_feedback(model, tokenizer, new_angle_differences)
print(f"New angle differences: {new_angle_differences}")
print(f"Generated feedback: {feedback}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Average training loss: 0.6832
Validation loss: 0.0324
Epoch 2/10, Average training loss: 0.0526
Validation loss: 0.0186
Epoch 3/10, Average training loss: 0.0371
Validation loss: 0.0136
Epoch 4/10, Average training loss: 0.0290
Validation loss: 0.0130
Epoch 5/10, Average training loss: 0.0265
Validation loss: 0.0131
Epoch 6/10, Average training loss: 0.0249
Validation loss: 0.0132
Epoch 7/10, Average training loss: 0.0237
Validation loss: 0.0133
Epoch 8/10, Average training loss: 0.0235
Validation loss: 0.0128
Epoch 9/10, Average training loss: 0.0231
Validation loss: 0.0129
Epoch 10/10, Average training loss: 0.0218


In [ ]:
new_angle_differences = np.random.randn(8)
feedback = generate_feedback(model, tokenizer, new_angle_differences)
print(f"New angle differences: {new_angle_differences}")
print(f"Generated feedback: {feedback}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


New angle differences: [-0.73645986 -0.13397846  0.07488161 -0.14690699  0.78306098  1.51854789
  0.4446968   0.12303562]
Generated feedback: 왼쪽 팔꿈치�


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import numpy as np

# 데이터셋 클래스 정의
class JointAngleDataset(Dataset):
    def __init__(self, angle_differences, feedbacks, tokenizer, max_length):
        self.angle_differences = angle_differences
        self.feedbacks = feedbacks
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.angle_differences)

    def __getitem__(self, idx):
        angle_str = ' '.join([f"{angle:.2f}" for angle in self.angle_differences[idx]])
        text = f"Angle differences: {angle_str} Feedback: {self.feedbacks[idx]}"
        encoding = self.tokenizer(text,
                                  truncation=True,
                                  max_length=self.max_length,
                                  padding='max_length',
                                  return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

# 가상의 데이터 생성 (더 다양한 피드백 추가)
x = [45.71, 32.08, 23.86, 16.81, 14.44, -21.22, 20.13, 10.23]
array_x = np.array(x)
angle_differences = np.tile(array_x, (1000, 1))  # 데이터 수를 늘림

feedbacks = [
    "오른쪽 팔꿈치와 왼쪽 팔꿈치, 오른쪽 무릎과 오른쪽 골반을 더 구부리고, 왼쪽 발목은 더 펴주세요",

    # ... 더 많은 피드백 (다양하게 추가)
] * 1000  # 데이터를 충분히 다양하게

# GPT-2 토크나이저 및 모델 로드
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')

# 데이터셋 및 데이터로더 생성
max_length = 128
dataset = JointAngleDataset(angle_differences, feedbacks, tokenizer, max_length)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 에폭 및 스케줄러 설정
num_epochs = 10  # 학습 에폭 수를 늘림
num_training_steps = num_epochs * len(train_dataloader)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# 학습 루프
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average training loss: {avg_train_loss:.4f}")

    # 검증
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Validation loss: {avg_val_loss:.4f}")

# 모델 저장
model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")

# 새로운 각도 차이에 대한 피드백 생성 함수
def generate_feedback(model, tokenizer, angle_differences, max_length=50):
    model.eval()
    angle_str = ' '.join([f"{angle:.2f}" for angle in angle_differences])
    input_text = f"Angle differences: {angle_str} Feedback:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

    with torch.no_grad():
        output = model.generate(input_ids,
                                max_length=max_length,
                                num_return_sequences=1,
                                do_sample=True,
                                temperature=0.7,
                                top_k=50,
                                top_p=0.95,
                                pad_token_id=tokenizer.eos_token_id)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text.split("Feedback:")[1].strip()

# 새로운 각도 차이에 대한 피드백 생성 예시
new_angle_differences = np.random.randn(8)
feedback = generate_feedback(model, tokenizer, new_angle_differences)
print(f"New angle differences: {new_angle_differences}")
print(f"Generated feedback: {feedback}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Average training loss: 0.4314
Validation loss: 0.0002
Epoch 2/10, Average training loss: 0.0112
Validation loss: 0.0000
Epoch 3/10, Average training loss: 0.0105
Validation loss: 0.0000
Epoch 4/10, Average training loss: 0.0095
Validation loss: 0.0000
Epoch 5/10, Average training loss: 0.0085
Validation loss: 0.0000
Epoch 6/10, Average training loss: 0.0069
Validation loss: 0.0000
Epoch 7/10, Average training loss: 0.0067
Validation loss: 0.0000
Epoch 8/10, Average training loss: 0.0058
Validation loss: 0.0000
Epoch 9/10, Average training loss: 0.0060
Validation loss: 0.0000
Epoch 10/10, Average training loss: 0.0048
Validation loss: 0.0000


The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


New angle differences: [-0.23564181 -0.59956449 -0.71752353  0.07528994  0.34908622 -1.60335107
  0.05858793  0.36366433]
Generated feedback: 오른쪽 팔꿈


In [1]:
import tensorflow as tf
from transformers import TFAutoModelForCausalLM, AutoTokenizer
import numpy as np

# KoGPT2 모델과 토크나이저 로드
model_name = 'skt/kogpt2-base-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForCausalLM.from_pretrained(model_name)

# 특별 토큰 추가
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

# 가상의 데이터 생성 (실제 프로젝트에서는 실제 데이터로 대체해야 함)
x = [45.71, 32.08, 23.86, 16.81, 14.44, -21.22, 20.13, 10.23]
array_x = np.array(x)
angle_differences = np.tile(array_x, (1000, 1))  # 데이터 수를 늘림
feedbacks = [
     "오른쪽 팔꿈치와 왼쪽 팔꿈치, 오른쪽 무릎과 오른쪽 골반을 더 구부리고, 왼쪽 발목은 더 펴주세요",
    # ... 더 많은 피드백 (1000개)
] * 1000  # 1000개의 샘플을 만들기 위해 1000번 반복

# 데이터 전처리 함수
def preprocess_data(angle_diffs, feedback_texts):
    inputs = []
    for angles, feedback in zip(angle_diffs, feedback_texts):
        angle_str = ' '.join([f"{angle:.2f}" for angle in angles])
        text = f"각도 차이: {angle_str} 피드백: {feedback}"
        inputs.append(text)

    encodings = tokenizer(inputs, truncation=True, padding=True, max_length=128, return_tensors="tf")
    return encodings

# 데이터 전처리
dataset = preprocess_data(angle_differences, feedbacks)

# TensorFlow 데이터셋 생성
tf_dataset = tf.data.Dataset.from_tensor_slices((
    dict(dataset),
    dataset["input_ids"]
))

# 데이터셋 분할 및 배치 설정
train_size = int(0.9 * len(tf_dataset))
tf_dataset = tf_dataset.shuffle(len(tf_dataset))
train_dataset = tf_dataset.take(train_size).batch(16)
val_dataset = tf_dataset.skip(train_size).batch(16)

# 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer])

# 학습
model.fit(train_dataset, validation_data=val_dataset, epochs=3)

# 모델 저장
model.save_pretrained("./fine_tuned_kogpt2_tf")
tokenizer.save_pretrained("./fine_tuned_kogpt2_tf")

# 새로운 각도 차이에 대한 피드백 생성 함수
def generate_feedback(model, tokenizer, angle_differences, max_length=50):
    angle_str = ' '.join([f"{angle:.2f}" for angle in angle_differences])
    input_text = f"각도 차이: {angle_str} 피드백:"
    input_ids = tokenizer.encode(input_text, return_tensors='tf')

    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1,
                            do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text.split("피드백:")[1].strip()

# 새로운 각도 차이에 대한 피드백 생성 예시
new_angle_differences = np.random.randn(8)
feedback = generate_feedback(model, tokenizer, new_angle_differences)
print(f"새로운 각도 차이: {new_angle_differences}")
print(f"생성된 피드백: {feedback}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

OSError: skt/kogpt2-base-v2 does not appear to have a file named tf_model.h5 but there is a file for PyTorch weights. Use `from_pt=True` to load this model from those weights.

In [4]:
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2LMHeadModel
import numpy as np
from transformers import PreTrainedTokenizerFast
import torch

# KoGPT2 모델과 토크나이저 로드
model_name = 'skt/kogpt2-base-v2'
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name, bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')
model = TFGPT2LMHeadModel.from_pretrained(model_name, from_pt=True)

# 특별 토큰 추가
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

# 가상의 데이터 생성 (실제 프로젝트에서는 실제 데이터로 대체해야 함)
x = [45.71, 32.08, 23.86, 16.81, 14.44, -21.22, 20.13, 10.23]
array_x = np.array(x)
angle_differences = np.tile(array_x, (1000, 1))  # 데이터 수를 늘림
feedbacks = [
     "오른쪽 팔꿈치와 왼쪽 팔꿈치, 오른쪽 무릎과 오른쪽 골반을 더 구부리고, 왼쪽 발목은 더 펴주세요",
    # ... 더 많은 피드백 (1000개)
] * 1000  # 1000개의 샘플을 만들기 위해 1000번 반복

# 데이터 전처리 함수
def preprocess_data(angle_diffs, feedback_texts):
    inputs = []
    for angles, feedback in zip(angle_diffs, feedback_texts):
        angle_str = ' '.join([f"{angle:.2f}" for angle in angles])
        text = f"각도 차이: {angle_str} 피드백: {feedback}"
        inputs.append(text)

    encodings = tokenizer(inputs, truncation=True, padding=True, max_length=128, return_tensors="tf")
    return encodings

# 데이터 전처리
dataset = preprocess_data(angle_differences, feedbacks)

# TensorFlow 데이터셋 생성
tf_dataset = tf.data.Dataset.from_tensor_slices((
    dict(dataset),
    dataset["input_ids"]
))

# 데이터셋 분할 및 배치 설정
train_size = int(0.9 * len(tf_dataset))
tf_dataset = tf_dataset.shuffle(len(tf_dataset))
train_dataset = tf_dataset.take(train_size).batch(16)
val_dataset = tf_dataset.skip(train_size).batch(16)

# 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer])

# 학습
model.fit(train_dataset, validation_data=val_dataset, epochs=3)

# 모델 저장
model.save_pretrained("./fine_tuned_kogpt2_tf")
tokenizer.save_pretrained("./fine_tuned_kogpt2_tf")


def generate_feedback(model, tokenizer, angle_differences, max_length=50):
    angle_str = ' '.join([f"{angle:.2f}" for angle in angle_differences])
    input_text = f"각도 차이: {angle_str} 피드백:"
    input_ids = tokenizer.encode(input_text, return_tensors='tf')

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text.split("피드백:")[1].strip() if "피드백:" in generated_text else generated_text

# 새로운 각도 차이에 대한 피드백 생성 예시
new_angle_differences = np.random.randn(8)
feedback = generate_feedback(model, tokenizer, new_angle_differences)
print(f"새로운 각도 차이: {new_angle_differences}")
print(f"생성된 피드백: {feedback}")

# 모델이 제대로 동작하는지 간단한 테스트
test_input = tokenizer("안녕하세요", return_tensors='tf')
test_output = model.generate(test_input.input_ids, max_length=50)
print("테스트 출력:", tokenizer.decode(test_output[0], skip_special_tokens=True))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.3.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initial

Epoch 1/3
57/57 [==============================] - 153s 2s/step - loss: 0.4433 - val_loss: 4.5850e-04
Epoch 2/3
57/57 [==============================] - 110s 2s/step - loss: 0.0013 - val_loss: 1.7930e-04
Epoch 3/3
57/57 [==============================] - 113s 2s/step - loss: 6.7118e-04 - val_loss: 1.0772e-04
새로운 각도 차이: [-1.4364661  -0.97345525 -1.39218831  0.76623488  1.00118871  1.7178369
 -0.94884208 -0.29302985]
생성된 피드백: ::19::18::2::1:;:/:-:ᆞ:2
테스트 출력: 안녕하세요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요요


In [5]:
import tensorflow as tf
from transformers import PreTrainedTokenizerFast, TFGPT2LMHeadModel
import numpy as np

# KoGPT2 모델과 토크나이저 로드
model_name = 'skt/kogpt2-base-v2'
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name, bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')
model = TFGPT2LMHeadModel.from_pretrained(model_name, from_pt=True)

# 가상의 학습 데이터 생성 (실제 프로젝트에서는 실제 데이터로 대체해야 함)
angle_differences = np.random.randint(-30, 31, size=(1000, 8))
feedbacks = [
    "오른쪽 손목은 더 구부려주시고 왼쪽 무릎은 더 펴주세요.",
    "왼쪽 팔꿈치를 더 구부리고 오른쪽 발목을 더 펴주세요.",
    "허리를 더 곧게 펴고 오른쪽 어깨를 낮춰주세요.",
    # ... 더 많은 피드백 (1000개)
] * 250  # 1000개의 샘플을 만들기 위해 250번 반복

# 데이터 전처리 함수
def preprocess_data(angle_diffs, feedback_texts):
    inputs = []
    outputs = []
    for angles, feedback in zip(angle_diffs, feedback_texts):
        angle_str = ' '.join([f"{angle}" for angle in angles])
        input_text = f"각도 차이: {angle_str}"
        inputs.append(input_text)
        outputs.append(feedback)

    input_encodings = tokenizer(inputs, truncation=True, padding=True, max_length=64, return_tensors="tf")
    output_encodings = tokenizer(outputs, truncation=True, padding=True, max_length=64, return_tensors="tf")
    return input_encodings, output_encodings

# 데이터 전처리
input_dataset, output_dataset = preprocess_data(angle_differences, feedbacks)

# TensorFlow 데이터셋 생성
tf_dataset = tf.data.Dataset.from_tensor_slices((
    dict(input_dataset),
    output_dataset["input_ids"]
))

# 데이터셋 분할 및 배치 설정
train_size = int(0.9 * len(tf_dataset))
tf_dataset = tf_dataset.shuffle(len(tf_dataset))
train_dataset = tf_dataset.take(train_size).batch(16)
val_dataset = tf_dataset.skip(train_size).batch(16)

# 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer])

# 학습
model.fit(train_dataset, validation_data=val_dataset, epochs=3)

# 모델 저장
model.save_pretrained("./fine_tuned_kogpt2_tf")
tokenizer.save_pretrained("./fine_tuned_kogpt2_tf")

# 새로운 각도 차이에 대한 피드백 생성 함수
def generate_feedback(model, tokenizer, angle_differences, max_length=50):
    angle_str = ' '.join([f"{angle}" for angle in angle_differences])
    input_text = f"각도 차이: {angle_str}"
    input_ids = tokenizer.encode(input_text, return_tensors='tf')

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text.replace(input_text, "").strip()

# 새로운 각도 차이에 대한 피드백 생성 예시
new_angle_differences = [16, 15, 23, 16, 24, 16, 12, 18]
feedback = generate_feedback(model, tokenizer, new_angle_differences)
print(f"새로운 각도 차이: {new_angle_differences}")
print(f"생성된 피드백: {feedback}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.3.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initial

Epoch 1/3


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-package

In [7]:
import tensorflow as tf
from transformers import PreTrainedTokenizerFast, TFGPT2LMHeadModel
import numpy as np

# KoGPT2 모델과 토크나이저 로드
model_name = 'skt/kogpt2-base-v2'
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name, bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')
model = TFGPT2LMHeadModel.from_pretrained(model_name, from_pt=True)

# 가상의 학습 데이터 생성 (실제 프로젝트에서는 실제 데이터로 대체해야 함)
angle_differences = np.random.randint(-30, 31, size=(1000, 8))
feedbacks = [
    "오른쪽 손목은 더 구부려주시고 왼쪽 무릎은 더 펴주세요.",
    "왼쪽 팔꿈치를 더 구부리고 오른쪽 발목을 더 펴주세요.",
    "허리를 더 곧게 펴고 오른쪽 어깨를 낮춰주세요.",
    # ... 더 많은 피드백 (1000개)
] * 250  # 1000개의 샘플을 만들기 위해 250번 반복

# 데이터 전처리 함수
def preprocess_data(angle_diffs, feedback_texts):
    inputs = []
    for angles, feedback in zip(angle_diffs, feedback_texts):
        angle_str = ' '.join([f"{angle}" for angle in angles])
        text = f"각도 차이: {angle_str} 피드백: {feedback}"
        inputs.append(text)

    encodings = tokenizer(inputs, truncation=True, padding=True, max_length=128, return_tensors="tf")
    input_ids = encodings['input_ids']
    attention_mask = encodings['attention_mask']

    # labels를 input_ids와 동일하게 설정
    labels = tf.identity(input_ids)

    # "피드백:" 이전의 토큰은 -100으로 설정 (손실 계산에서 무시됨)
    for i, input_text in enumerate(inputs):
        feedback_start = input_text.index("피드백:")
        feedback_start_token = len(tokenizer.encode(input_text[:feedback_start]))
        labels = tf.tensor_scatter_nd_update(
            labels,
            [[i, j] for j in range(feedback_start_token)],
            tf.constant([-100] * feedback_start_token)
        )

    return input_ids, attention_mask, labels

# 데이터 전처리
input_ids, attention_mask, labels = preprocess_data(angle_differences, feedbacks)

# TensorFlow 데이터셋 생성
tf_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": input_ids, "attention_mask": attention_mask},
    labels
))

# 데이터셋 분할 및 배치 설정
train_size = int(0.9 * len(tf_dataset))
tf_dataset = tf_dataset.shuffle(len(tf_dataset))
train_dataset = tf_dataset.take(train_size).batch(16)
val_dataset = tf_dataset.skip(train_size).batch(16)

# 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer)

# 학습
model.fit(train_dataset, validation_data=val_dataset, epochs=3)

# 모델 저장
model.save_pretrained("./fine_tuned_kogpt2_tf")
tokenizer.save_pretrained("./fine_tuned_kogpt2_tf")

# 새로운 각도 차이에 대한 피드백 생성 함수
def generate_feedback(model, tokenizer, angle_differences, max_length=50):
    angle_str = ' '.join([f"{angle}" for angle in angle_differences])
    input_text = f"각도 차이: {angle_str} 피드백:"
    input_ids = tokenizer.encode(input_text, return_tensors='tf')

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text.split("피드백:")[1].strip()

# 새로운 각도 차이에 대한 피드백 생성 예시
new_angle_differences = [16, 15, 23, 16, 24, 16, 12, 18]
feedback = generate_feedback(model, tokenizer, new_angle_differences)
print(f"새로운 각도 차이: {new_angle_differences}")
print(f"생성된 피드백: {feedback}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.3.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initial

Epoch 1/3
43/43 [==============================] - 115s 2s/step - loss: 0.5164 - val_loss: 0.0537
Epoch 2/3
43/43 [==============================] - 72s 2s/step - loss: 0.0560 - val_loss: 0.0674
Epoch 3/3
43/43 [==============================] - 72s 2s/step - loss: 0.0598 - val_loss: 0.0524
새로운 각도 차이: [16, 15, 23, 16, 24, 16, 12, 18]
생성된 피드백: 오른쪽 손목은 더 구부려주시고 왼쪽 무릎은 더 펴주세요. 으 어진 다 분만 펴고 오른쪽 무릎도 더 펴서주십시오 순


In [11]:
import tensorflow as tf
from transformers import PreTrainedTokenizerFast, TFGPT2LMHeadModel
import numpy as np

# KoGPT2 모델과 토크나이저 로드
model_name = 'skt/kogpt2-base-v2'
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name, bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')
model = TFGPT2LMHeadModel.from_pretrained(model_name, from_pt=True)

# 임의로 생성한 5개의 각도 차이 배열과 그에 따른 피드백
angle_differences = [
    [16, -12, 23, -8, 5, -10, 18, -7],
    [-20, 15, -18, 10, -5, 8, -12, 6],
    [8, -6, 12, -15, 20, -18, 7, -9],
    [-14, 10, -7, 5, -22, 17, -8, 13],
    [19, -11, 25, -13, 6, -9, 14, -16]
]

feedbacks = [
    "오른쪽 팔꿈치를 더 구부려주세요. 왼쪽 팔꿈치를 더 펴주세요. 오른쪽 무릎을 더 구부려주세요. 왼쪽 무릎을 더 펴주세요. 오른쪽 발목을 더 구부려주세요. 왼쪽 발목을 더 펴주세요. 오른쪽 골반을 더 구부려주세요. 왼쪽 골반을 더 펴주세요.",
    "오른쪽 팔꿈치를 더 펴주세요. 왼쪽 팔꿈치를 더 구부려주세요. 오른쪽 무릎을 더 펴주세요. 왼쪽 무릎을 더 구부려주세요. 오른쪽 발목을 더 펴주세요. 왼쪽 발목을 더 구부려주세요. 오른쪽 골반을 더 펴주세요. 왼쪽 골반을 더 구부려주세요.",
    "오른쪽 팔꿈치를 더 구부려주세요. 왼쪽 팔꿈치를 더 펴주세요. 오른쪽 무릎을 더 구부려주세요. 왼쪽 무릎을 더 펴주세요. 오른쪽 발목을 더 구부려주세요. 왼쪽 발목을 더 펴주세요. 오른쪽 골반을 더 구부려주세요. 왼쪽 골반을 더 펴주세요.",
    "오른쪽 팔꿈치를 더 펴주세요. 왼쪽 팔꿈치를 더 구부려주세요. 오른쪽 무릎을 더 펴주세요. 왼쪽 무릎을 더 구부려주세요. 오른쪽 발목을 더 펴주세요. 왼쪽 발목을 더 구부려주세요. 오른쪽 골반을 더 펴주세요. 왼쪽 골반을 더 구부려주세요.",
    "오른쪽 팔꿈치를 더 구부려주세요. 왼쪽 팔꿈치를 더 펴주세요. 오른쪽 무릎을 더 구부려주세요. 왼쪽 무릎을 더 펴주세요. 오른쪽 발목을 더 구부려주세요. 왼쪽 발목을 더 펴주세요. 오른쪽 골반을 더 구부려주세요. 왼쪽 골반을 더 펴주세요."
]

# 원래 코드에서 이 부분을 수정합니다:
angle_differences = np.array(angle_differences)
feedbacks = feedbacks * 200  # 1000개의 샘플을 만들기 위해 200번 반복

# 데이터 전처리 함수
def preprocess_data(angle_diffs, feedback_texts):
    inputs = []
    for angles, feedback in zip(angle_diffs, feedback_texts):
        angle_str = ' '.join([f"{angle}" for angle in angles])
        text = f"각도 차이: {angle_str} 피드백: {feedback}"
        inputs.append(text)

    encodings = tokenizer(inputs, truncation=True, padding=True, max_length=128, return_tensors="tf")
    input_ids = encodings['input_ids']
    attention_mask = encodings['attention_mask']

    labels = tf.identity(input_ids)

    for i, input_text in enumerate(inputs):
        feedback_start = input_text.index("피드백:")
        feedback_start_token = len(tokenizer.encode(input_text[:feedback_start]))
        labels = tf.tensor_scatter_nd_update(
            labels,
            [[i, j] for j in range(feedback_start_token)],
            tf.constant([-100] * feedback_start_token)
        )

    return input_ids, attention_mask, labels

# 데이터 전처리
input_ids, attention_mask, labels = preprocess_data(angle_differences, feedbacks)

# TensorFlow 데이터셋 생성
tf_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": input_ids, "attention_mask": attention_mask},
    labels
))

# 데이터셋 분할 및 배치 설정
train_size = int(0.9 * len(tf_dataset))
tf_dataset = tf_dataset.shuffle(len(tf_dataset))
train_dataset = tf_dataset.take(train_size).batch(16)
val_dataset = tf_dataset.skip(train_size).batch(16)

# 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer)

# 학습
model.fit(train_dataset, validation_data=val_dataset, epochs=10)

# 모델 저장
model.save_pretrained("./fine_tuned_kogpt2_tf")
tokenizer.save_pretrained("./fine_tuned_kogpt2_tf")

# 새로운 각도 차이에 대한 피드백 생성 함수
def generate_feedback(model, tokenizer, angle_differences, max_length=100):
    angle_str = ' '.join([f"{angle}" for angle in angle_differences])
    input_text = f"각도 차이: {angle_str} 피드백:"
    input_ids = tokenizer.encode(input_text, return_tensors='tf')

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    feedback = generated_text.split("피드백:")[1].strip()

    return feedback

# 새로운 각도 차이에 대한 피드백 생성 예시
new_angle_differences = [16, -15, 23, -16, 24, -16, 12, -18]
feedback = generate_feedback(model, tokenizer, new_angle_differences)
print(f"새로운 각도 차이: {new_angle_differences}")
print(f"생성된 피드백: {feedback}")

# 입력값에 따른 기대 피드백 출력 (비교용)
print("\n기대되는 피드백:")
expected_feedback = []
for i, angle in enumerate(new_angle_differences):
    if angle > 0:
        expected_feedback.append(f"{joints[i]}를 더 구부려주세요.")
    elif angle < 0:
        expected_feedback.append(f"{joints[i]}를 더 펴주세요.")
print(" ".join(expected_feedback))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.3.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initial

Epoch 1/10
1/1 [==============================] - 43s 43s/step - loss: 1.6290 - val_loss: 1.2482
Epoch 2/10
1/1 [==============================] - 1s 1s/step - loss: 1.3125 - val_loss: 0.6106
Epoch 3/10
1/1 [==============================] - 1s 1s/step - loss: 0.6381 - val_loss: 0.4685
Epoch 4/10
1/1 [==============================] - 1s 1s/step - loss: 0.5056 - val_loss: 0.4667
Epoch 5/10
1/1 [==============================] - 1s 1s/step - loss: 0.5087 - val_loss: 0.2649
Epoch 6/10
1/1 [==============================] - 1s 1s/step - loss: 0.2751 - val_loss: 0.3949
Epoch 7/10
1/1 [==============================] - 1s 1s/step - loss: 0.4161 - val_loss: 0.3118
Epoch 8/10
1/1 [==============================] - 1s 1s/step - loss: 0.3301 - val_loss: 0.2229
Epoch 9/10
1/1 [==============================] - 1s 1s/step - loss: 0.2369 - val_loss: 0.2776
Epoch 10/10
1/1 [==============================] - 1s 1s/step - loss: 0.2973 - val_loss: 0.2507
새로운 각도 차이: [16, -15, 23, -16, 24, -16, 12, -18]